<a href="https://colab.research.google.com/github/PradyumnaKrishna/Colab-Hacks/blob/RDP-v2/Colab%20RDP/Colab%20RDP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Colab RDP** : Remote Desktop to Colab Instance

Used Google Remote Desktop & Ngrok Tunnel

> **Warning : Not for Cryptocurrency Mining<br></br>** 
>**Why are hardware resources such as T4 GPUs not available to me?** The best available hardware is prioritized for users who use Colaboratory interactively rather than for long-running computations. Users who use Colaboratory for long-running computations may be temporarily restricted in the type of hardware made available to them, and/or the duration that the hardware can be used for. We encourage users with high computational needs to use Colaboratory’s UI with a local runtime. Please note that using Colaboratory for cryptocurrency mining is disallowed entirely, and may result in being banned from using Colab altogether.

Google Colab can give you Instance with 12GB of RAM and GPU for 12 hours (Max.) for Free users. Anyone can use it to perform Heavy Tasks.

To use other similiar Notebooks use my Repository **[Colab Hacks](https://github.com/PradyumnaKrishna/Colab-Hacks)**

In [ ]:
#@title **Create User**
#@markdown Enter Username and Password

username = "user" #@param {type:"string"}
password = "root" #@param {type:"string"}

print("Creating User and Setting it up")

# Creation of user
! sudo useradd -m $username &> /dev/null

# Add user to sudo group
! sudo adduser $username sudo &> /dev/null
    
# Set password of user to 'root'
! echo '$username:$password' | sudo chpasswd

# Change default shell from sh to bash
! sed -i 's/\/bin\/sh/\/bin\/bash/g' /etc/passwd

print("User Created and Configured")

In [ ]:
#@title **RDP**
#@markdown  It takes 4-5 minutes for installation

#@markdown  Visit http://remotedesktop.google.com/headless and Copy the command after authentication

CRP = "" #@param {type:"string"}

#@markdown  To Sleep the Colab Notebook

Sleep = False #@param {type:"boolean"}

def CRD():
    with open('install.sh', 'w') as script:
        script.write("""#! /bin/bash

b='\033[1m'
r='\E[31m'
g='\E[32m'
c='\E[36m'
endc='\E[0m'
enda='\033[0m'

printf "\n\n$c$b    Loading Installer $endc$enda" >&2
if sudo apt-get update &> /dev/null
then
    printf "\r$g$b    Installer Loaded $endc$enda\n" >&2
else
    printf "\r$r$b    Error Occured $endc$enda\n" >&2
    exit
fi

printf "\n$g$b    Installing Chrome Remote Desktop $endc$enda" >&2
{
    wget https://dl.google.com/linux/direct/chrome-remote-desktop_current_amd64.deb
    sudo dpkg --install chrome-remote-desktop_current_amd64.deb
    sudo apt install --assume-yes --fix-broken
} &> /dev/null &&
printf "\r$c$b    Chrome Remote Desktop Installed $endc$enda\n" >&2 ||
{ printf "\r$r$b    Error Occured $endc$enda\n" >&2; exit; }
sleep 3

printf "$g$b    Installing Desktop Environment $endc$enda" >&2
{
    sudo DEBIAN_FRONTEND=noninteractive \
        apt install --assume-yes xfce4 desktop-base
    sudo bash -c 'echo "exec /etc/X11/Xsession /usr/bin/xfce4-session" > /etc/chrome-remote-desktop-session'  
    sudo apt install --assume-yes xscreensaver
    sudo systemctl disable lightdm.service
} &> /dev/null &&
printf "\r$c$b    Desktop Environment Installed $endc$enda\n" >&2 ||
{ printf "\r$r$b    Error Occured $endc$enda\n" >&2; exit; }
sleep 3

printf "$g$b    Installing Google Chrome $endc$enda" >&2
{
    wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
    sudo dpkg --install google-chrome-stable_current_amd64.deb
    sudo apt install --assume-yes --fix-broken
} &> /dev/null &&
printf "\r$c$b    Google Chrome Installed $endc$enda\n" >&2 ||
printf "\r$r$b    Error Occured $endc$enda\n" >&2
sleep 3

printf "$g$b    Installing other Tools $endc$enda" >&2
if sudo apt install nautilus nano -y &> /dev/null
then
    printf "\r$c$b    Other Tools Installed $endc$enda\n" >&2
else
    printf "\r$r$b    Error Occured $endc$enda\n" >&2
fi
sleep 3

printf "\n$g$b    Installation Completed $endc$enda\n\n" >&2""")

    ! chmod +x install.sh
    ! ./install.sh

    # Adding user to CRP group
    ! sudo adduser $username chrome-remote-desktop &> /dev/null

    # Finishing Work
    ! su - $username -c """$CRP"""

    print("Finished Succesfully")

    if Sleep:
        ! sleep 43200

try:
    if username:
        if CRP == "" :
            print("Please enter authcode from the given link")
        else:
            CRD()
except NameError:
    print("username variable not found")
    print("Create a User First")

In [ ]:
#@title **Google Drive Mount**
#@markdown Google Drive used as Persistance HDD for files.<br>
#@markdown Mounted at `user` Home directory inside drive folder.


def MountGDrive():
    mountpoint = "/home/"+username+"/drive"

    from os import environ as env
    from os.path import join as path_join
    from google.colab import drive

    env['user'] = username
    env['user_home'] = "/home/{}".format(username)
    env['mount_script'] = path_join(env['user_home'], 'drive_mount.sh')

    ! runuser -l $user -c "yes | python3 -m pip install --user google-colab"  > /dev/null 2>&1

    ! cp -r /root/.config/Google /home/$user/.config/Google > /dev/null 2>&1
    ! mkdir -p /home/$user/.config/Google/DriveFS/Logs
    ! chown -R $user:$user /home/user/.config/Google/DriveFS/Logs
    ! mkdir -p $mountpoint > /dev/null 2>&1
    ! chown -R $user:$user /content/.config
    ! chown -R $user:$user /content > /dev/null 2>&1

    mount = """
#!/bin/sh

from os import environ as env
from google.colab import drive

env['CLOUDSDK_CONFIG']  = '/content/.config'
drive.mount('{}')""".format(mountpoint)

    with open(path_join(env['user_home'], "drive_mount.sh"), "w") as script:
        script.write(mount)

    ! chown -R  $user $user_home
    ! chmod u+x $mount_script

    ! runuser -l $user -c "python3 /home/$user/drive_mount.sh"

    ! chown $user:$user $mountpoint > /dev/null 2>&1

try:
    if username:
        MountGDrive()
except NameError:
    print("username variable not found")
    print("Create a user first")

In [ ]:
#@title **SSH** (Using NGROK)

! pip install colab_ssh --upgrade &> /dev/null
from colab_ssh import launch_ssh, init_git
from IPython.display import clear_output

#@markdown Copy authtoken from https://dashboard.ngrok.com/auth
ngrokToken = "" #@param {type:'string'}

def runNGROK():
    launch_ssh(ngrokToken, password)
    clear_output()

    print("ssh", username, end='@')
    ! curl -s http://localhost:4040/api/tunnels | python3 -c \
            "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'][6:].replace(':', ' -p '))"

try:
    if username:
        pass
    elif password:
        pass
except NameError:
    print("No user found using username and password as 'root'")
    username='root'
    password='root'

if ngrokToken == "":
    print("No ngrokToken Found, Please enter it")
else:
    runNGROK()

In [ ]:
#@title **Colab Shutdown**

#@markdown To Kill NGROK Tunnel
NGROK = False #@param {type:'boolean'}

#@markdown To Unmount GDrive
GDrive = False #@param {type:'boolean'}

if NGROK:
    ! killall ngrok

if GDrive:
    from google.colab import drive
    drive.flush_and_unmount()